<a href="https://colab.research.google.com/github/Prathvirajshett/raj-shetty-/blob/master/date%20and%20valid_email%20pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 74 kB/s 
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58762 sha256=191dfc6f6f1200c40c9d33d3bbcd60f1c3c87d9623e6dca1d211787483f76658
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark


In [3]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 63.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=da5900d5d1c3d6c065c33f761cc3316c79f0038c9b417f3ebd97a6ec60f316c8
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
import pyspark

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [6]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

In [7]:
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [8]:
!pip install -q findspark

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


In [10]:
import findspark
findspark.init()


In [11]:
findspark.find()


'/content/spark-3.2.1-bin-hadoop3.2'

In [12]:
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [14]:
spark

In [16]:
df=spark.read.csv("/content/data_prep.csv",header=True,inferSchema=True)

In [17]:
df.show()

+------+----------------+----------+------+-----------+
|userid|         emailid|      date|amount|   discount|
+------+----------------+----------+------+-----------+
|     1| test1@gmail.com|01-01-2019|   100|        1.0|
|     2| test2@gmail.com|02-01-2019|   100|        1.0|
|     3| test3@gmail.com|03-01-2019|   101|        2.0|
|     4| test4@gmail.com|04-01-2019|   102|       10.0|
|     5| test5@gmail.com|05-01-2019| 102.5|        2.0|
|     6| test6@gmail.com|06-01-2019| 103.2|        1.0|
|     7| test7@gmail.com|07-01-2019| 103.9|        3.0|
|     8| test8@gmail.com|08-01-2019| 104.6|        5.0|
|     9| test9@gmail.com|09-01-2019| 105.3|4.571428571|
|    10|test10@gmail.com|10-01-2019|   106|4.892857143|
|    11|test11@gmail.com|11-01-2019| 106.7|5.214285714|
|    12|test12@gmail.com|12-01-2019| 107.4|5.535714286|
|    13|test13@gmail.com|13-01-2019| 108.1|5.857142857|
|    14|test14@gmail.com|14-01-2019| 108.8|6.178571429|
|    15|test15@gmail.com|15-01-2019| 109.5|     

In [20]:
df.count()

902

In [19]:
df1=df.dropna()

In [21]:
df1.count()

901

In [22]:
df.filter("emailid is NULL AND date is NULL AND amount is NULL").show() 

+------+-------+----+------+--------+
|userid|emailid|date|amount|discount|
+------+-------+----+------+--------+
+------+-------+----+------+--------+



In [23]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

In [24]:
import datetime

In [54]:
def vali_date(i):

  try:
    result=datetime.datetime.strptime(i,'%d-%m-%Y')
    new_reult=result.datetime.strftime('%d-%m-%Y')
    return new_reult
  except AttributeError:
            pass
  raise ValueError('no valid date format found')



In [51]:
vali_date_UDF= udf(lambda i:vali_date(i))

In [60]:


df.select(col('date'),vali_date_UDF(col("date").alias("date_temp"))).show()

PythonException: ignored

In [30]:
df2=df.select("discount").distinct().show()

+-----------+
|   discount|
+-----------+
|       15.5|
|31.57142857|
|42.17857143|
|       8.75|
|17.10714286|
|      31.25|
|        0.0|
|10.67857143|
|21.60714286|
|11.96428571|
|20.32142857|
|27.07142857|
|18.71428571|
|38.96428571|
|14.85714286|
|32.53571429|
|      26.75|
|44.42857143|
|      35.75|
|8.107142857|
+-----------+
only showing top 20 rows



In [39]:
from datetime import datetime

def try_parsing_date(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy'):
        try:
            date_object=datetime.strptime(text, fmt)
            new_object=date_object.strftime('%Y-%m-%d')
            
            return new_object
            
        except ValueError:
            pass
    raise ValueError('no valid date format found')
#ab=try_parsing_date('12-10-1996')

In [42]:
my_udf=udf(lambda x :try_parsing_date(x))

In [45]:
abc=df.select(col('date'),my_udf(col('date').alias("new_data")))

In [46]:
abc.show()

+----------+--------------------------+
|      date|<lambda>(date AS new_data)|
+----------+--------------------------+
|01-01-2019|                2019-01-01|
|02-01-2019|                2019-01-02|
|03-01-2019|                2019-01-03|
|04-01-2019|                2019-01-04|
|05-01-2019|                2019-01-05|
|06-01-2019|                2019-01-06|
|07-01-2019|                2019-01-07|
|08-01-2019|                2019-01-08|
|09-01-2019|                2019-01-09|
|10-01-2019|                2019-01-10|
|11-01-2019|                2019-01-11|
|12-01-2019|                2019-01-12|
|13-01-2019|                2019-01-13|
|14-01-2019|                2019-01-14|
|15-01-2019|                2019-01-15|
|16-01-2019|                2019-01-16|
|17-01-2019|                2019-01-17|
|18-01-2019|                2019-01-18|
|19-01-2019|                2019-01-19|
|20-01-2019|                2019-01-20|
+----------+--------------------------+
only showing top 20 rows



In [63]:
UDF=udf(lambda text:try_parsing_date(text),)

In [62]:
ddf=df.select((col("date")).alias("date_temp")).show()

+----------+
| date_temp|
+----------+
|01-01-2019|
|02-01-2019|
|03-01-2019|
|04-01-2019|
|05-01-2019|
|06-01-2019|
|07-01-2019|
|08-01-2019|
|09-01-2019|
|10-01-2019|
|11-01-2019|
|12-01-2019|
|13-01-2019|
|14-01-2019|
|15-01-2019|
|16-01-2019|
|17-01-2019|
|18-01-2019|
|19-01-2019|
|20-01-2019|
+----------+
only showing top 20 rows



In [61]:
pip install validate_email

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for validate-email: filename=validate_email-1.3-py3-none-any.whl size=5482 sha256=b4f80cda273f409e7c211c582264946d52db38e95e9e142835823da533b2f6d4
  Stored in directory: /root/.cache/pip/wheels/ff/8f/92/c43287715852eaa75e0d8aa1941c481072b4a82c4f4975074e
Successfully built validate-email


In [64]:
from validate_email import validate_email

In [67]:
is_valid=validate_email('exampleexample@gmail.com')

In [68]:
print(is_valid)

True


In [69]:
def email_parser(str1):
  is_valid=validate_email(str1)
  if is_valid:
    return str1
  else:
       "ERORR"

In [70]:
email_udf=udf(lambda x:email_parser(x))

In [73]:
ddd=df.select(col('emailid'),email_udf(col('emailid')).alias('new_emailid'))

In [74]:
ddd.show()

+----------------+----------------+
|         emailid|     new_emailid|
+----------------+----------------+
| test1@gmail.com| test1@gmail.com|
| test2@gmail.com| test2@gmail.com|
| test3@gmail.com| test3@gmail.com|
| test4@gmail.com| test4@gmail.com|
| test5@gmail.com| test5@gmail.com|
| test6@gmail.com| test6@gmail.com|
| test7@gmail.com| test7@gmail.com|
| test8@gmail.com| test8@gmail.com|
| test9@gmail.com| test9@gmail.com|
|test10@gmail.com|test10@gmail.com|
|test11@gmail.com|test11@gmail.com|
|test12@gmail.com|test12@gmail.com|
|test13@gmail.com|test13@gmail.com|
|test14@gmail.com|test14@gmail.com|
|test15@gmail.com|test15@gmail.com|
|test16@gmail.com|test16@gmail.com|
|test17@gmail.com|test17@gmail.com|
|test18@gmail.com|test18@gmail.com|
|test19@gmail.com|test19@gmail.com|
|test20@gmail.com|test20@gmail.com|
+----------------+----------------+
only showing top 20 rows



In [77]:

e_edf=ddd.filter(col('new_emailid')=='ERROR')

In [78]:
e_edf.show()

+-------+-----------+
|emailid|new_emailid|
+-------+-----------+
+-------+-----------+



In [80]:
cd=df.dropDuplicates()

In [82]:
cd.count()

901